In [1]:
# 모든 ascii 문자로 사전 만들기
import string
all_chars = string.printable

vocab_size = len(all_chars)
vocab_dict = dict((c, i) for (i, c) in enumerate(all_chars))

# 문자열을 수치 리스트로 변환하는 함수
def str2ints(s, vocab_dict):
    return [vocab_dict[c] for c in s]
  
# 수치 리스트를 문자열로 변환하는 함수
def ints2str(x, vocab_array):
    return "".join([vocab_array[i] for i in x])

In [3]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                           DataLoader,
                           TensorDataset)
import tqdm

In [4]:
class ShakespeareDataset(Dataset):
    def __init__(self, path, chunk_size=200):
        # 파일을 읽어서 수치 리스트로 변환
        data = str2ints(open(path).read().strip(), vocab_dict)
        
        # Tensor로 변환해서 split 한다
        data = torch.tensor(data, dtype=torch.int64).split(chunk_size)
        
        # 마지막 덩어리(chunk)의 길이를 확인해서 부족한 경우 버린다後のchunkの長さをチェックして足りない場合には捨てる
        if len(data[-1]) < chunk_size:
            data = data[:-1]
        
        self.data = data
        self.n_chunks = len(self.data)
    
    def __len__(self):
        return self.n_chunks

    def __getitem__(self, idx):
        return self.data[idx]

In [5]:
ds = ShakespeareDataset("input.txt",  chunk_size=200)
loader = DataLoader(ds, batch_size=32, shuffle=True, num_workers=4)

In [8]:
class SequenceGenerationNet(nn.Module):
    def __init__(self, num_embeddings, 
                 embedding_dim=50, 
                 hidden_size=50,
                 num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_size,
                            num_layers,
                            batch_first=True,
                            dropout=dropout)
        # Linear의 output 크기는 첫 Embedding의 
        # input 크기와 같은 num_embeddings
        self.linear = nn.Linear(hidden_size, num_embeddings)
        
    def forward(self, x, h0=None):
        x = self.emb(x)
        x, h = self.lstm(x, h0)
        x = self.linear(x)
        return x, h

In [9]:

def generate_seq(net, start_phrase="The King said ",
                 length=200, temperature=0.8, device="cpu"):
    # 모델을 평가 모드로 설정
    net.eval()
    # 출력 수치를 저장할 리스트
    result = []
    
    # 시작 문자열을 Tensor로 변환
    start_tensor = torch.tensor(
        str2ints(start_phrase, vocab_dict),
        dtype=torch.int64
    ).to(device)
    # 선두에 batch 차원을 붙인다
    x0 = start_tensor.unsqueeze(0) 
    # RNN을 통해서 출력과 새로운 내부 상태를 얻는다
    o, h = net(x0)
    # 출력을 정규화돼있지 않은 확률로 변환
    out_dist = o[:, -1].view(-1).exp()
    # 확률로부터 실제 문자의 인덱스를 샘플링グ
    top_i = torch.multinomial(out_dist, 1)[0]
    # 결과 저장
    result.append(top_i)
    
    # 생성된 결과를 차례로 RNN에 넣는다
    for i in range(length):
        inp = torch.tensor([[top_i]], dtype=torch.int64)
        inp = inp.to(device)
        o, h = net(inp, h)
        out_dist = o.view(-1).exp()
        top_i = torch.multinomial(out_dist, 1)[0]
        result.append(top_i)
    
    # 시작 문자열과 생성된 문자열을 모아서 반환
    return start_phrase + ints2str(result, all_chars)

In [10]:
from statistics import mean
net = SequenceGenerationNet(vocab_size, 20, 50,
                            num_layers=2, dropout=0.1)
net.to("cuda:0")
opt = optim.Adam(net.parameters())
# 다중 식별 문제이므로 SoftmaxCrossEntropyLoss가 손실 함수가 된다
loss_f = nn.CrossEntropyLoss()

for epoch in range(50):
    net.train()
    losses = []
    for data in tqdm.tqdm(loader):
        # x는 처음부터 마지막의 하나 앞 문자까지
        x = data[:, :-1]
        # y는 두 번째부터 마지막 문자까지
        y = data[:, 1:]
        
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        
        y_pred, _ = net(x)
        # batch와 step 축을 통합해서 CrossEntropyLoss에 전달
        loss = loss_f(y_pred.view(-1, vocab_size), y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    # 현재 손실 함수와 생성된 문장 예 표시
    print(epoch, mean(losses))
    with torch.no_grad():
        print(generate_seq(net, device="cuda:0"))

100%|██████████| 175/175 [00:07<00:00, 24.18it/s]


0 3.4976481737409317
The King said or asA m edemoe!naoongbmh it Udr mahhr'e yupn  wa ooidynkaea ,naep:ru Haefdd nrihO etee
t Nd  c.i]tinf
rod bnii reicrede eohceaeyu o to nn
rted
tr   :rase,seeHTmd 
tita o
io eouhem, loy a SERnwce


100%|██████████| 175/175 [00:07<00:00, 24.50it/s]


1 3.082128519330706
The King said uoe,.
Bh:esOr Aom ns
ehsfrnu, hue sp?hiuay ina,s.uhr owe tar ant, awe, tihe
j
J?it tehmrf tsa Mninpiuet iean tctfkr shadan, wa, nrun gooe erot
monen, oee 
Tor'l'e naptcouP

wiyoop non IA
p an om
moyhl 


100%|██████████| 175/175 [00:07<00:00, 24.87it/s]


...

47 1.693846616744995
The King said you for my confers!
And so your ack to scilly in baved of all squeet cemsted?

BROTANTES:
Now your beaused. Whas' he knax it basts and broon if a soults in the ere varged: upon fire of his jest;
'Fisel


100%|██████████| 175/175 [00:07<00:00, 24.69it/s]


48 1.69193781375885
The King said him on waste thou arged fore this hest
than 'spile of all
And allows is my back;
To you fol Nathiching enest in this loved his very;
To which corment as you fe to just cirkandly world hance with not ga


100%|██████████| 175/175 [00:07<00:00, 24.59it/s]


49 1.688188762664795
The King said make have her kneever trays
I'll tompt, sightous contay,
Thy offon it true upon,
As they stonk
Though wouss: Llight Bort, one take think, and thou heart,
But with rymippy
But but son
And do creach upon
